In [140]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [141]:
company_list = ['MayoClinic_10_8']  #keyword
#webMD_10_8,'ClevelandClinic_10_8','MayoClinic_10_8'
startID_list = ['917032337422118914'] #twitter ID  https://twitter.com/MikeLevinCA/status/913394885352349704
#917041903790690304,'917041647602556930','917032337422118914'

In [142]:
# First test with neurodiversity microsoft since:2015-04-30 until:2017-04-30
# https://twitter.com/search?l=en&q=neurodiversity%20microsoft%20%20since%3A2015-01-01%20until%3A2017-05-31&src=typd&lang=en
def connectTwitter(company,startID):
    #url_list = ['https://twitter.com/WebMD', 'https://twitter.com/ClevelandClinic', 'https://twitter.com/MayoClinic']
    url = 'https://twitter.com/MayoClinic'
    param = {
        'vertical':'default',
        #'q':'since:2017-09-01 until:2017-10-01 ',# since:2015-01-01 until:2017-05-31 #time
        'l':'en',
        'src':'typd',
        'include_available_features':'1',
        'include_entities':'1',
        'lang':'en',
        'max_position': startID,
        'reset_error_state':'false'
    } # TWEET-857-857 means starting with the first page.
    header = {
        'accept':'application/json, text/javascript, */*; q=0.01',
        'accept-encoding':'gzip, deflate, sdch, br',
        'accept-language':'en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4',
        'dnt':'1',
        'referer':'https://twitter.com/MayoClinic?l=&q=&src=typd&vertical=&max_position=&',# Remove the information that
                                                                                           # param has included.
        'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'x-requested-with':'XMLHttpRequest',
        'x-twitter-active-user':'yes'
    }
    retry = 0;
    while retry < 3:
        try:
            response = requests.get(url,params=param, headers=header)
            html = response.content.decode('utf-8')
            soup = BeautifulSoup(html,'lxml')

            tweets_html_id = soup.find_all('div',class_='stream')[0].find_all('li')
            tweets_id = [tweet_html_id.get('data-item-id') for tweet_html_id in tweets_html_id]
            tweets_id = [x for x in tweets_id if x is not None]
            next_endID = tweets_id[len(tweets_id)-1]

        except IndexError:
            retry = retry + 1
            if retry == 3:
                soup = None
                next_endID = None
            else:
                time.sleep(60 * retry)
                continue
        break

    return soup,next_endID,tweets_id

In [143]:
# DOM Parser
def parser_DF(soup):
    tweets_html = soup.find_all('li',class_='js-stream-item stream-item stream-item ') # note of the space
    
    # get twitter ID
    tweets_id = [tweet_html.find('div').get('data-tweet-id') 
                           for tweet_html in tweets_html]
    # get user name
    tweets_username = [tweet_html.find('div').get('data-screen-name') 
                           for tweet_html in tweets_html]
    # get DataTime
    tweets_datetime = []
    tweets_datetime_html = [tweet_html.find('small',class_='time') for tweet_html in tweets_html]
    for tweet_datetime_html in tweets_datetime_html:
        if tweet_datetime_html is not None:
            tweets_datetime.append(tweet_datetime_html.find('a').get('title'))
        else:
            tweets_datetime.append(None)
            
    # get content
    tweets_content = [tweet_html.find('p',class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text.strip()
                      if tweet_html.find('p',class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text') is not None
                      else None for tweet_html in tweets_html]
    # get replies, retweets, likes
    tweets_replies = [tweet_html.find('span',class_='ProfileTweet-action--reply u-hiddenVisually').find('span').get('data-tweet-stat-count') 
                      if tweet_html.find('span',class_='ProfileTweet-action--reply u-hiddenVisually') is not None
                      else None for tweet_html in tweets_html]
    tweets_retweet = [tweet_html.find('span',class_='ProfileTweet-action--retweet u-hiddenVisually').find('span').get('data-tweet-stat-count') 
                      if tweet_html.find('span',class_='ProfileTweet-action--retweet u-hiddenVisually') is not None
                      else None for tweet_html in tweets_html]
    tweets_likes = [tweet_html.find('span',class_='ProfileTweet-action--favorite u-hiddenVisually').find('span').get('data-tweet-stat-count') 
                    if tweet_html.find('span',class_='ProfileTweet-action--favorite u-hiddenVisually') is not None
                    else None for tweet_html in tweets_html]
    # get image link
    tweets_image=[]
    tweets_image_html = [tweet_html.find('div',class_='AdaptiveMedia-singlePhoto') for tweet_html in tweets_html]
    for tweet_image_html in tweets_image_html:
        if tweet_image_html is not None:
            tweets_image.append(tweet_image_html.find('div').get('data-image-url'))
        else:
            tweets_image.append(None)
            
    # get image of article link        
    tweets_a_image=[]
    tweets_a_image_html = [tweet_html.find('div',class_='js-media-container') for tweet_html in tweets_html]
    for tweet_a_image_html in tweets_a_image_html:
        if tweet_a_image_html is not None:
            tweets_a_image.append(tweet_a_image_html.find('div').get('data-card-url'))
        else:
            tweets_a_image.append(None) 
            
    # get video link        
    tweets_video=[]
    tweets_video_html = [tweet_html.find('div',class_='AdaptiveMedia-videoContainer') for tweet_html in tweets_html]
    for tweet_video_html in tweets_video_html:
        if tweet_video_html is not None:
            tweets_video.append(tweet_video_html.find('div').get('class'))
        else:
            tweets_video.append(None)     
    
    # get verified account
    #tweets_verified=[]
    #tweets_html[6].find('span',class_='Icon Icon--verified')
    #tweets_verified_html = [tweet_html.find('span',class_='Icon Icon--verified') for tweet_html in tweets_html]
    #for tweet_verified_html in tweets_verified_html:
        #if tweet_verified_html is None:
            #tweets_verified.append(0)
        #else:
            #tweets_verified.append(1)
    
    # store data to dataframe        
    tweets = [('username',tweets_username),
              ('TwitterID',tweets_id),
              #('verified',tweets_verified),
              ('DateTime',tweets_datetime),
              ('content',tweets_content),
              ('likes',tweets_likes),
              ('retweet',tweets_retweet),
              ('replies',tweets_replies),
              ('image',tweets_image),
              ('articleImage', tweets_a_image),
              ('video',tweets_video)]
            

    tweets_df = pd.DataFrame.from_items(tweets)
    
    return tweets_df

In [144]:
# Automatically get lastID and go to next page
companies_html = [] # global variable
for i in range(len(company_list)): # for loop of company_list
    soup_list=[]
    id_list=[]

    if len(id_list)==0: # First page: startID and endID would be the same
        s = connectTwitter(company_list[i],startID_list[i])
        soup_list.append(s[0])
        id_list.append(s[1])

    if len(id_list)>=1:
        while(id_list[-1] is not None):
            s = connectTwitter(company_list[i],id_list[-1])
            soup_list.append(s[0])
            id_list.append(s[1])

    if(id_list[-1] is None): # because of the try catch in the def
        del soup_list[-1]
        del id_list[-1]
        
    companies_html.append(soup_list)

In [145]:
# parse html and merge dataframe ---CSV
companies_df_list = [] # global variable
for j in range(len(companies_html)):
    tweets_df_list = []
    i=0
    for soup in companies_html[j]:
        print(i)
        tweets_df = parser_DF(soup)
        tweets_df_list.append(tweets_df)
        i=i+1
    companies_df_list.append(pd.concat(tweets_df_list,ignore_index=True))
    companies_df_list[j].to_csv(company_list[j]+'.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148


In [132]:
tweets_df

,username,TwitterID,DateTime,content,likes,retweet,replies,image,articleImage,video
0,MayoClinic,843920329995182080,1:21 PM - 20 Mar 2017,Eight years of education comes down to one day...,12,4,0,None,None,None
1,MayoProceedings,843893155682574340,11:33 AM - 20 Mar 2017,CME offered: Discovering new treatments for #G...,15,13,0,https://pbs.twimg.com/media/C7YcWi-U0AAqCAS.jpg,None,None


In [133]:
tweets_df_list

[]

In [123]:
len(companies_df_list[0])

2950